In [6]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Engine
from typing import Optional, List, Dict, Any
import pandas as pd
import os
import yaml
from pathlib import Path
from dotenv import load_dotenv

load_dotenv(".env")

def load_env(env_file: str = "../../.env") -> None:
    """
    Load environment variables from a .env file.
    Works in both script and notebook environments.
    """

    env_path = Path(env_file)

    if not env_path.is_absolute():
        # Script
        if "__file__" in globals():
            project_root = Path(__file__).resolve().parents[1]
        # Notebook
        else:
            project_root = Path.cwd()

        env_path = project_root / env_path

    if not env_path.exists():
        raise FileNotFoundError(f"❌ .env file not found: {env_path}")
    
    # load_dotenv(env_path)

def fct_load_config(config_filename: str = "config.yaml") -> dict:
    """
    Goal
    ----
    Load configuration parameters from a YAML file.

    This function works both in:
    - Python scripts
    - Jupyter notebooks (where __file__ is not defined)

    Parameters
    ----------
    config_filename : str
        Relative or absolute path to the YAML configuration file.

    Returns
    -------
    dict
        Dictionary containing configuration parameters.
    """

    config_path = Path(config_filename)

    # Cas 1 : chemin absolu → on l'utilise directement
    if config_path.is_absolute():
        final_path = config_path

    else:
        # Cas 2 : Script Python (__file__ existe)
        if "__file__" in globals():
            project_root = Path(__file__).resolve().parents[1]

        # Cas 3 : Notebook Jupyter (__file__ n'existe pas)
        else:
            project_root = Path(os.getcwd())

        final_path = project_root / config_path

    if not final_path.exists():
        raise FileNotFoundError(f"❌ Config file not found: {final_path}")

    with open(final_path, "r", encoding="utf-8") as f:
        config = yaml.safe_load(f)

    return config


def create_postgres_engine(
    host: str,
    database: str,
    user: str,
    password: str,
    port: int = 5432
) -> Optional[Engine]:
    """
    Crée un moteur SQLAlchemy pour PostgreSQL.

    Paramètres
    ----------
    host : str
        Adresse du serveur PostgreSQL
    database : str
        Nom de la base de données
    user : str
        Utilisateur PostgreSQL
    password : str
        Mot de passe
    port : int, optionnel
        Port PostgreSQL (par défaut 5432)

    Retour
    ------
    sqlalchemy.engine.Engine | None
        Moteur SQLAlchemy ou None en cas d'erreur
    """
    try:
        engine = create_engine(
            f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}",
            pool_pre_ping=True
        )
        return engine
    except Exception as e:
        print(f"❌ Erreur de création du moteur SQLAlchemy : {e}")
        return None


def execute_select(
    engine: Engine,
    query: str,
    params: dict | None = None
) -> List[Dict[str, Any]]:
    """
    Exécute une requête SELECT et retourne les résultats sous forme de dictionnaires.
    """
    with engine.connect() as conn:
        result = conn.execute(text(query), params or {})
        return [dict(row._mapping) for row in result]
    
def select_to_dataframe(
    engine: Engine,
    query: str,
    params: dict | None = None
) -> pd.DataFrame:
    """
    Exécute une requête SELECT et retourne un DataFrame pandas.
    """
    return pd.read_sql(text(query), engine, params=params)


def execute_query(
    engine: Engine,
    query: str,
    params: dict | None = None
) -> None:
    """
    Exécute une requête INSERT, UPDATE ou DELETE.
    """
    try:
        with engine.begin() as conn:  # commit automatique
            conn.execute(text(query), params or {})
    except Exception as e:
        print(f"❌ Erreur SQL : {e}")


def dataframe_to_table(
    df: pd.DataFrame,
    table_name: str,
    engine: Engine,
    schema: str = "public",
    if_exists: str = "append"
) -> None:
    """
    Insère un DataFrame pandas dans une table PostgreSQL.
    """
    df.to_sql(
        table_name,
        engine,
        schema=schema,
        if_exists=if_exists,
        index=False,
        method="multi"
    )


In [7]:
host=os.getenv("HOST")
database=os.getenv("DATABASE")
user=os.getenv("USER")
password=os.getenv("PASSWORD")


engine = create_postgres_engine(
    host=host,
    database=database,
    user=user,
    password=password
)

In [16]:
req = """
WITH winners AS (
    SELECT
        home_team,
        away_team,
        CASE
            WHEN home_result > away_result THEN home_team
            WHEN away_result > home_result THEN away_team
            ELSE 'Draw'
        END AS winner,
        CASE
            WHEN home_result > away_result THEN away_team   
            WHEN away_result > home_result THEN home_team
            ELSE 'Draw'
        END AS loser
    FROM matches)
SELECT winner, COUNT(*) AS win_count, 
       ROUND( (COUNT(*) * 100.0) / (SELECT COUNT(*) FROM winners WHERE winner <> 'Draw'), 2) AS win_percentage
FROM winners
WHERE winner <> 'Draw'
GROUP BY winner
ORDER BY win_count DESC;
"""

In [17]:
# SELECT
df_matches = select_to_dataframe(
    engine,
    req
)
df_matches

,winner,win_count,win_percentage
0,France,15,9.20
1,Germany,12,7.36
2,Argentina,12,7.36
3,Belgium,12,7.36
4,Brazil,10,6.13
5,Netherlands,10,6.13
6,Colombia,7,4.29
7,Croatia,7,4.29
8,Uruguay,7,4.29
9,England,6,3.68


In [18]:
req = """
SELECT * 
FROM matches;
"""

# SELECT
df_matches = select_to_dataframe(
    engine,
    req
)
df_matches

,match_id,date,home_team,away_team,home_result,away_result,stage,edition,city
0,1,20140612170000,Brazil,Croatia,3,1,group_a,2014,Sao Paulo
1,2,20140613130000,Mexico,Cameroon,1,0,group_a,2014,Natal
2,3,20140613160000,Spain,Netherlands,1,5,group_b,2014,Salvador
3,4,20140613180000,Chile,Australia,3,1,group_b,2014,Cuiaba
4,5,20140614130000,Colombia,Greece,3,0,group_c,2014,Belo Horizonte
...,...,...,...,...,...,...,...,...,...
203,204,20221210200000,England,France,1,2,Quarter-final,None,None
204,205,20221213200000,Argentina,Croatia,3,0,Semi-final,None,None
205,206,20221214200000,France,Morocco,2,0,Semi-final,None,None
206,207,20221217160000,Croatia,Morocco,2,1,Play-off for third place,None,None
